# Emoji generation using LSTM

<b><i>Importing python libraries</i></b>

In [1]:
import numpy as np
from emo_utils import *
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, Reshape, Lambda
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)

Using TensorFlow backend.


<b><i>Reading dataset in train and test varibles</i></b>

In [4]:
X_train, Y_train = read_csv('data/train_emoji.csv')
X_test, Y_test = read_csv('data/tesss.csv')

In [6]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(132,)
(132,)
(56,)
(56,)


<b><i>Maximum length of the longest sentence in the dataset</i></b>

In [7]:
maxLen = len(max(X_train, key=len).split())
maxLen

10

<b><i>Converting the train and test label to their one hot vector form matrix</i></b>

In [8]:
Y_oh_train = convert_to_one_hot(Y_train, C = 5)
Y_oh_test = convert_to_one_hot(Y_test, C = 5)
print(Y_oh_train.shape)

(132, 5)


<b><i>Function to read and store the glove embedding matrix</i></b>

In [10]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r', encoding="utf8") as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

<b><i>Reading the glove embedding matrix</i></b>

In [11]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('data/glove.6B.50d.txt')

<b><i>Function convert every sentence to its corresponding indices vector using word_to_index dictionary; it also pads zero if incase the sentence vector is less than the max length of the sentence </i></b>

In [12]:
def sentences_to_indices(X, word_to_index, max_len):
 
    m = X.shape[0]                                   
    X_indices = np.zeros((m, max_len))
    for i in range(m):                               
        
        sentence_words = X[i].lower().split()
        j = 0
        
        for w in sentence_words:
            X_indices[i, j] = word_to_index[w] 
            j = j + 1
    
    return X_indices

<b><i>Converting our train data to trainable form</i></b>

In [14]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 5)
print(X_train_indices.shape)

(132, 10)


<b><i>Function builds the <u>Embedding()</u> layer in Keras. After this layer is built, we can pass the output of <u>sentences_to_indices()</u> to it as an input, and the Embedding() layer will return the word embeddings for that sentence.</i></b>

In [15]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):

    vocab_len = len(word_to_index) + 1                  
    emb_dim = word_to_vec_map["cucumber"].shape[0]
    
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    embedding_layer = Embedding(vocab_len, emb_dim, trainable = False)

    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

<b><i>Model Function</i></b>

In [17]:
def Emojify_V2(input_shape, word_to_vec_map, word_to_index):

    sentence_indices = Input(shape = input_shape, dtype = 'int32')
    
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)
    print("Shape of embeddings: ", embeddings.shape)
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state; the returned output should be a batch of sequences.
    X = LSTM(128, return_sequences=True)(embeddings)
    X = Dropout(0.5)(X)
    
    # Propagate X trough another LSTM layer with 128-dimensional hidden state; the returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128, return_sequences=False)(X)
    
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(5)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    
    model = Model(inputs = sentence_indices, outputs = X)
    
    return model

<b><i>Calling the Model Function</i></b>

In [18]:
model = Emojify_V2((maxLen,), word_to_vec_map, word_to_index)

Shape of embeddings:  (?, 10, 50)


<b><i>Compiling the model</i></b>

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

<b><i>Fitting our training data to the compiled model</i></b>

In [20]:
model.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 32, shuffle=True)

Epoch 1/50
132/132 [==============================] - 4s 32ms/step - loss: 1.5992 - acc: 0.1894
Epoch 2/50
132/132 [==============================] - 0s 3ms/step - loss: 1.5292 - acc: 0.2727
Epoch 3/50
132/132 [==============================] - 0s 3ms/step - loss: 1.4779 - acc: 0.3030
Epoch 4/50
132/132 [==============================] - 0s 3ms/step - loss: 1.4160 - acc: 0.3712
Epoch 5/50
132/132 [==============================] - 0s 3ms/step - loss: 1.3201 - acc: 0.5076
Epoch 6/50
132/132 [==============================] - 0s 2ms/step - loss: 1.1929 - acc: 0.6061
Epoch 7/50
132/132 [==============================] - 0s 2ms/step - loss: 1.0334 - acc: 0.6742
Epoch 8/50
132/132 [==============================] - 0s 2ms/step - loss: 0.9036 - acc: 0.6591
Epoch 9/50
132/132 [==============================] - 0s 2ms/step - loss: 0.8983 - acc: 0.6818
Epoch 10/50
132/132 [==============================] - 0s 2ms/step - loss: 0.8099 - acc: 0.6970A: 0s - loss: 0.7056 - acc: 0.73
Epoch 11/50
132/

<b><i>Converting our test data to testable form and evaluating our model</i></b>

In [21]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
Y_test_oh = convert_to_one_hot(Y_test, C = 5)
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print("Test accuracy = ", acc)

56/56 [==============================] - 1s 14ms/step
Test accuracy =  0.7678571343421936


<b><i>This code allows you to see the mislabelled examples</i></b>

In [24]:
C = 5
y_test_oh = np.eye(C)[Y_test.reshape(-1)]
X_test_indices = sentences_to_indices(X_test, word_to_index, maxLen)
pred = model.predict(X_test_indices)
for i in range(len(X_test)):
    x = X_test_indices
    num = np.argmax(pred[i])
    if(num != Y_test[i]):
        print('Expected emoji:'+ label_to_emoji(Y_test[i]) + ' prediction: '+ X_test[i] + label_to_emoji(num).strip())

Expected emoji:😄 prediction: she got me a nice present	❤️
Expected emoji:😞 prediction: work is hard	😄
Expected emoji:😞 prediction: This girl is messing with me	❤️
Expected emoji:😞 prediction: work is horrible	😄
Expected emoji:😞 prediction: she is a bully	❤️
Expected emoji:😞 prediction: go away	⚾
Expected emoji:😞 prediction: yesterday we lost again	⚾


<b><i>Here, we can see that know the below example is giving the correct label</i></b>

In [23]:
x_test = np.array(['not feeling happy'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
print(x_test[0] +' '+  label_to_emoji(np.argmax(model.predict(X_test_indices))))

not feeling happy 😞
